In [0]:
from os import listdir
import numpy as np
from numpy import expand_dims
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from PIL import Image
import cv2
import math
from scipy import ndimage
from keras.models import load_model

In [0]:
SUBMISSION_IMAGES = "./SubmissionImages"

In [0]:
# define a configuration for the model
class LicensePlateConfig(Config):
	# define the name of the configuration
	NAME = "LicensePlate_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 1
	# number of training steps per epoch
	STEPS_PER_EPOCH = 131
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

In [0]:
# prepare config
config = LicensePlateConfig()

In [0]:
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=config)

In [0]:
model.load_weights("/content/drive/My Drive/LicensePlateProject/licenseplate_cfg20200509T2207/mask_rcnn_licenseplate_cfg_0031.h5", by_name=True)

In [0]:
#Detect plate on submission images
imagesToPredict = [image for image in listdir(SUBMISSION_IMAGES)]
#show prediction for 30 images
for i in range(len(imagesToPredict)-200):
    pyplot.figure(figsize=(15,20))
    # load the image 
    image = pyplot.imread(SUBMISSION_IMAGES+"/"+imagesToPredict[i])
    # convert pixel values (e.g. center)
    scaled_image = mold_image(image, config)
    # convert image into one sample
    sample = expand_dims(scaled_image, 0)
    # make prediction
    yhat = model.detect(sample, verbose=0)[0]
    # define subplot
    #pyplot.subplot(len(imagesToPredict), 2, i*2+1)
    # plot raw pixel data
    pyplot.imshow(image)
    pyplot.title('Predicted')
    ax = pyplot.gca()
    # plot each box
    for box in yhat['rois']:
        # get coordinates
        y1, x1, y2, x2 = box
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
	# show the figure
    pyplot.show()
    pyplot.savefig("pred"+str(i)+".jpeg")

In [0]:
#Read image
image = pyplot.imread("/content/drive/My Drive/LicensePlateProject/SubmissionImages/1000.jpg")
# convert pixel values (e.g. center)
scaled_image = mold_image(image, config)
# convert image into one sample
sample = expand_dims(scaled_image, 0)
# make prediction
yhat = model.detect(sample, verbose=0)[0]

In [0]:
def firstCrop(img, predictions):
    firstCrop = []
    for box in predictions['rois']:
        # get coordinates
        y1, x1, y2, x2 = box
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        if width < height:
            continue
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='red')
        xtop = x1
        ytop = y1
        xbottom = x2
        ybottom = y2
        firstCrop.append(img[ytop:ybottom, xtop:xbottom])
        cv2.rectangle(img,(xtop,ytop),(xbottom,ybottom),(0,255,0),0)
    return firstCrop

In [0]:
l = firstCrop(image, yhat)

pyplot.figure(figsize=(15,20))
pyplot.imshow(l[0])
pyplot.show()

In [0]:
def secondCrop(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,127,255,0)
    contours,_ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    if(len(areas)!=0):
        max_index = np.argmax(areas)
        cnt=contours[max_index]
        x,y,w,h = cv2.boundingRect(cnt)
        bounds = cv2.boundingRect(cnt)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),0)
        secondCrop = img[y:y+h,x:x+w]
    else:
        secondCrop = img
    return secondCrop

In [0]:

a = secondCrop(l[0])
pyplot.imshow(a)

cv2.imwrite("crop.jpg",a)

In [0]:
im = cv2.imread("/content/drive/My Drive/LicensePlateProject/crop.jpg", cv2.IMREAD_GRAYSCALE)
(_, gray) = cv2.threshold(im, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
gray = 255 - gray
ctrs,_ = cv2.findContours(gray.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
j=0
digits = []
for rect in rects:
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    crop_im = im[y:y+h, x:x+w]
    if w > h:
        continue
    if h < 15 or h < 30:
        continue
    digits.append(crop_im)
    pyplot.figure(figsize=(3,3))
    pyplot.imshow(crop_im)
    pyplot.show()


In [0]:
i = 0
for im in digits:
    cv2.imwrite("seg_"+str(i)+".jpg", im)
    i += 1

In [0]:
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

i = 0
for im in listdir('./PlateDigits/'):
    gray = cv2.imread('./PlateDigits/'+im, cv2.IMREAD_GRAYSCALE)
    gray = cv2.resize(gray, (28, 28))  # 255-gray for inverting black and white
    (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    flatten = gray.flatten() / 255.0

    while np.sum(gray[0]) == 0:
        gray = gray[1:]

    while np.sum(gray[:,0]) == 0:
        gray = np.delete(gray,0,1)

    while np.sum(gray[-1]) == 0:
        gray = gray[:-1]

    while np.sum(gray[:,-1]) == 0:
        gray = np.delete(gray,-1,1)

    rows,cols = gray.shape
    if rows > cols:
        factor = 20.0/rows
        rows = 20
        cols = int(round(cols*factor))
        gray = cv2.resize(gray, (cols,rows))
    else:
        factor = 20.0/cols
        cols = 20
        rows = int(round(rows*factor))
        gray = cv2.resize(gray, (cols, rows))

    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

    shiftx,shifty = getBestShift(gray)
    shifted = shift(gray,shiftx,shifty)
    gray = shifted
    pyplot.imshow(gray)
    cv2.imwrite("prepro_"+str(i)+".jpg", gray)
    i = i + 1

In [0]:
model = load_model('./model_02.h5')

In [0]:
result = []
s = '-'
for im in listdir('./Prepro/'):
    gray = cv2.imread('./Prepro/'+im, cv2.IMREAD_GRAYSCALE)
    img = gray.reshape(1,1,28,28).astype('float32')
    img = img / 255
    prediction = model.predict_classes(img)
    result.append(str(prediction[0]))
print(s.join(result))